In [536]:
import pandas as pd
from sdv.lite import TabularPreset
from sdv.evaluation import evaluate
import json
from collections import OrderedDict
import numpy as np
import math

In [537]:
config_file = open('config.json')
temp = json.load(config_file, object_pairs_hook=OrderedDict)
config = pd.DataFrame.from_dict(temp)
data_file = config["Input File"][0]
target_file = config["Output File"][0]
data = pd.read_csv(data_file)
headers = data.columns
features = []
for feature in config["Features"].keys():
    features.append(feature)
allowed_percentage = config["Percentage"][0]
number_of_samples = int(config["N_samples"][0])
types = config["Features"]

#print(features)


['Cancer Type Detailed', 'Diagnosis Age', 'IDH1', 'Mutation Count', 'Neoplasm Histologic Grade', 'Oncotree Code', 'Overall Survival (Months)', 'PABPC3', 'PTEN', 'Sample Type', 'TERT', 'Treatment Details before PDX']


In [538]:

#Remove unspecified columns
for header in headers:
    if not header in features:
        data.pop(header)
        #print("Deleting column", header, "because of configuration.")

#Remove columns with to many NaNs
headers = data.columns
for header in headers:
    nan_percentage = data[header].isna().sum()/len(data[header].index)
    if nan_percentage > allowed_percentage:
        data.pop(header)
        headers = headers.drop(header)
        print("Deleting column", header, "because of" ,"{:.2%}".format(nan_percentage), "NaNs.")

#removing unwanted nans and interpolating others
categoricals = []
numericals = []
for i in range(len(features)):
    if features[i] in headers:
        if types[i] == "numerical":
            numericals.append(features[i])
        if types[i] == "categorical":
            categoricals.append(features[i])

categoricals_that_need_work =[]

for categorical in categoricals:
    if (data[categorical].isna().any()):
        categoricals_that_need_work.append(categorical)

data.dropna(axis = 0, how='any', subset = numericals, inplace= True)

for numerical in numericals:
    data[numerical] = pd.to_numeric(data[numerical], errors='coerce')

data.interpolate(method='pad')






Deleting column Neoplasm Histologic Grade because of 91.28% NaNs.
Deleting column Treatment Details before PDX because of 90.58% NaNs.


,Cancer Type Detailed,Mutation Count,Oncotree Code,Overall Survival (Months),Sample Type,Diagnosis Age,TERT,IDH1,PABPC3,PTEN
0,Glioblastoma Multiforme,173.0,GBM,30.00,Recurrence,40,Thalidomide,Carboplatin,WT,WT
1,Glioblastoma Multiforme,1756.0,GBM,21.00,Recurrence,67,WT,WT,E2101K,P191T I195V
2,Glioblastoma Multiforme,158.0,GBM,8.04,Primary,62,WT,WT,K312R T319I,I101T
3,Glioblastoma Multiforme,161.0,GBM,25.92,Primary,64,WT,WT,V293G A313V K312R A154G K156R,WT
4,Glioblastoma Multiforme,669.0,GBM,44.88,Recurrence,65,WT,WT,WT,A313Rfs*14 F335Lfs*19 M326Gfs*21 K231E P191T E...
...,...,...,...,...,...,...,...,...,...,...
855,Anaplastic Oligodendroglioma,12.0,AODG,12.90,Recurrence,64,Promoter,WT,NS,WT
856,Anaplastic Oligoastrocytoma,14.0,AOAST,54.80,Recurrence,30,Promoter,R132H,NS,WT
857,Anaplastic Oligodendroglioma,7.0,AODG,93.90,Recurrence,45,Promoter,R132H,NS,WT
858,Anaplastic Oligodendroglioma,5.0,AODG,48.30,Recurrence,42,Promoter,R132H,NS,WT


In [539]:
#create validation set

validation_set = data.sample(frac=0.3)
validation_set.reset_index()


,index,Cancer Type Detailed,Mutation Count,Oncotree Code,Overall Survival (Months),Sample Type,Diagnosis Age,TERT,IDH1,PABPC3,PTEN
0,838,Anaplastic Astrocytoma,30.0,AASTR,47.00,Recurrence,36,WT,R132H,WT,WT
1,22,Glioblastoma Multiforme,124.0,GBM,21.48,Recurrence,52,Bevacizumab,Carboplatin,WT,WT
2,307,Glioblastoma Multiforme,8.0,GBM,17.50,Primary,68,Promoter,WT,NS,P95S
3,252,Glioblastoma Multiforme,4.0,GBM,33.70,Primary,52,Promoter,WT,NS,WT
4,544,Oligodendroglioma,4.0,ODG,198.00,Recurrence,35,Promoter,R132H,NS,WT
...,...,...,...,...,...,...,...,...,...,...,...
253,508,Anaplastic Astrocytoma,3.0,AASTR,84.00,Primary,45,WT,R132C,NS,WT
254,579,Glioblastoma Multiforme,2.0,GBM,43.90,Recurrence,60,Promoter,WT,NS,WT
255,510,Glioblastoma Multiforme,3.0,GBM,7.53,Recurrence,69,Promoter,WT,NS,WT
256,219,Glioblastoma Multiforme,2.0,GBM,6.35,Primary,76,WT,WT,NS,WT


In [540]:
#Generating Metadata

metadata = {}
metadata["fields"] = {}
for i in range(len(features)):
    if features[i] in headers:
        metadata["fields"][features[i]] = {}
        metadata["fields"][features[i]]["type"] = types[i]
        if sub_types[i] == sub_types[i]:
            metadata["fields"][features[i]]["subtype"] = sub_types[i]

metadata["constraints"] = []

#print(metadata)



IndexError: string index out of range

In [ ]:
#fit a model to the data and generating data
#model = TabularPreset(name='FAST_ML', metadata=metadata)
model = TabularPreset(name='FAST_ML', metadata=metadata)

model.fit(data)

new_data = model.sample(num_rows=number_of_samples)

new_data = new_data.round(decimals = 2)

new_data.to_csv(target_file, index=False)

print("The generated data is", "{:.2%}".format(evaluate(new_data, validation_set, metrics=['KSTest'])), "accurate to the original data. If this is unsatisfactory try using less or different features, more data or removing more NaNs.")

MetadataError: Invalid type and subtype combination for field Cancer Type Detailed: (c, None)